In [65]:
import os
import json
import boto3

import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

ACCESS_KEY = os.getenv("ID_KEY_ACCESS")
ACCESS_KEY_SECRETE = os.getenv("KEY_SECRET")
BUCKET_NAME = os.getenv("BUCKETNAME")

Python-dotenv could not parse statement starting at line 5


In [66]:
df_1 = pd.read_json("./hotel_1.json")
df_2 = pd.read_json("./hotel_2.json")
df_3 = pd.read_json("./temps.json")

In [67]:
# merge des 3 dataframe
def merge_outer(data_1:pd.DataFrame, data_2:pd.DataFrame, on:str) -> pd.DataFrame:
    df = pd.merge(data_1, data_2, how='outer', on=on)
    return df

df = merge_outer(df_1, df_2, "hotel")
df = merge_outer(df, df_3, "id")

In [68]:
# drop des hotel ou il manque la note
df.dropna()

,id,ville,hotel,url,note,description,latitutde,longitude,jour_1,jour_2,jour_3,jour_4,jour_5,lat,long
0,1,Le Mont-Saint-Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0",Vous pouvez bénéficier d'une réduction Genius ...,48.614700,-1.509617,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
1,1,Le Mont-Saint-Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2",Installé dans des espaces verts à seulement 2 ...,48.614247,-1.510545,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
2,1,Le Mont-Saint-Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,0",Vous pouvez bénéficier d'une réduction Genius ...,48.635085,-1.510540,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
3,1,Le Mont-Saint-Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1",L'hôtel De La Digue est un établissement tradi...,48.616882,-1.510918,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
4,1,Le Mont-Saint-Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3",Vous pouvez bénéficier d'une réduction Genius ...,48.635349,-1.510379,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,35,"1er arr., Lyon",Hamac Suites - Le Plâtre - Place des Terreaux,https://www.booking.com/hotel/fr/hamac-suites-...,"8,7","Doté d'une connexion Wi-Fi gratuite, le Hamac ...",45.766106,4.833282,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043
871,35,"7e arr., Lyon",B&B HOTEL LYON Centre Perrache Berthelot,https://www.booking.com/hotel/fr/b-amp-b-lyon-...,"7,8","Ouvert en décembre 2017, le B&B HOTEL LYON Cen...",45.747809,4.835540,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043
872,35,"2e arr., Lyon",Hôtel Charlemagne,https://www.booking.com/hotel/fr/hotelbestwest...,"7,8",Vous pouvez bénéficier d'une réduction Genius ...,45.746023,4.823894,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043
873,35,"5e arr., Lyon",MiHotel Tour Rose,https://www.booking.com/hotel/fr/mihotel-tour-...,"8,4",Vous pouvez bénéficier d'une réduction Genius ...,45.762725,4.826820,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043


In [5]:
# sauvegarde du dataframe en csv pour envoie dans un s3
csv = df.to_csv()
df.to_csv("hotel.csv")

In [6]:
# création d'une session aws
session = boto3.Session(aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=ACCESS_KEY_SECRETE)
s3 = session.resource("s3")

In [7]:
# création d'un bucket s3
bucket = s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [8]:
# ajout du fichier csv dans le compartiment bucket s3
put_object = bucket.put_object(Key="hotel.csv", Body=csv)

In [9]:
# recuperation du fichier csv dans un s3
df = pd.read_csv(f's3://{BUCKET_NAME}/hotel.csv')

In [76]:
df_temps = df[["id", "ville", "lat", "long"]]

In [77]:
df_temps["temperature_jour_1"] = df["jour_1"].apply(lambda x : x[0])
df_temps["temperature_jour_2"] = df["jour_2"].apply(lambda x : x[0])
df_temps["temperature_jour_3"] = df["jour_3"].apply(lambda x : x[0])
df_temps["temperature_jour_4"] = df["jour_4"].apply(lambda x : x[0])
df_temps["temperature_jour_5"] = df["jour_5"].apply(lambda x : x[0])
df_temps["temps_jour_1"] = df["jour_1"].apply(lambda x : x[1])
df_temps["temps_jour_2"] = df["jour_2"].apply(lambda x : x[1])
df_temps["temps_jour_3"] = df["jour_3"].apply(lambda x : x[1])
df_temps["temps_jour_4"] = df["jour_4"].apply(lambda x : x[1])
df_temps["temps_jour_5"] = df["jour_5"].apply(lambda x : x[1])

C:\Users\jerem\AppData\Local\Temp\ipykernel_6196\2982697953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps["temperature_jour_1"] = df["jour_1"].apply(lambda x : x[0])
C:\Users\jerem\AppData\Local\Temp\ipykernel_6196\2982697953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps["temperature_jour_2"] = df["jour_2"].apply(lambda x : x[0])
C:\Users\jerem\AppData\Local\Temp\ipykernel_6196\2982697953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [82]:
df_temps.drop_duplicates("id", inplace=True)
df_temps.reset_index(drop=True, inplace=True)

C:\Users\jerem\AppData\Local\Temp\ipykernel_6196\3048149287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps.drop_duplicates("id", inplace=True)


In [83]:
ad = df_temps[["temperature_jour_1", "temperature_jour_2", "temperature_jour_3", "temperature_jour_4", "temperature_jour_5"]]

In [84]:
df_temps["moyenne"] = ad.mean(axis=1).round(2)

C:\Users\jerem\AppData\Local\Temp\ipykernel_6196\4015349174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps["moyenne"] = ad.mean(axis=1).round(2)


In [85]:
df_temps

,id,ville,lat,long,temperature_jour_1,temperature_jour_2,temperature_jour_3,temperature_jour_4,temperature_jour_5,temps_jour_1,temps_jour_2,temps_jour_3,temps_jour_4,temps_jour_5,moyenne
0,1,Le Mont-Saint-Michel,48.635523,-1.510257,6.33,7.44,5.41,4.74,4.21,nuageux,légère pluie,peu nuageux,ciel dégagé,ciel dégagé,5.63
1,2,"Intra muros, Saint-Malo",48.649518,-2.026041,6.15,8.28,5.99,5.32,5.40,couvert,légère pluie,peu nuageux,ciel dégagé,peu nuageux,6.23
2,3,Bayeux,49.276462,-0.702474,6.08,7.44,3.68,3.92,4.11,couvert,légère pluie,légère pluie,nuageux,nuageux,5.05
3,4,"Perret, Le Havre",49.493898,0.107973,6.33,7.93,3.45,3.73,3.39,couvert,légère pluie,légère pluie,partiellement nuageux,ciel dégagé,4.97
4,5,"Centre-ville de Rouen, Rouen",49.440459,1.093966,5.85,6.88,3.67,3.41,2.25,couvert,légère pluie,couvert,ciel dégagé,ciel dégagé,4.41
5,6,"19e arr., Paris",48.858890,2.320041,4.83,6.46,4.80,2.92,1.44,couvert,nuageux,nuageux,ciel dégagé,ciel dégagé,4.09
6,7,Amiens,49.894171,2.295695,6.44,6.75,3.34,2.73,2.02,couvert,nuageux,couvert,ciel dégagé,ciel dégagé,4.26
7,8,"Vauban, Lille",50.636565,3.063528,6.46,7.23,4.32,2.45,2.07,légère pluie,légère pluie,partiellement nuageux,ciel dégagé,ciel dégagé,4.51
8,9,Strasbourg,48.584614,7.750713,3.94,4.69,3.96,2.46,1.68,couvert,nuageux,couvert,couvert,légère pluie,3.35
9,10,Orschwiller,48.249490,7.344296,1.00,2.27,1.13,-0.54,-0.44,partiellement nuageux,partiellement nuageux,couvert,couvert,légères chutes de neige,0.68


In [86]:
df.drop(["lat", "long","jour_1", "jour_2", "jour_3", "jour_4", "jour_5"], inplace=True, axis=1)

In [98]:
# engine = create_engine(f"postgresql+psycopg2://{YOUR_USERNAME}:{YOUR_PASSWORD}@{YOUR_HOSTNAME}/postgres", echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)

In [99]:
df.to_sql("hotel", engine)


2023-01-03 15:03:19,506 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hotel")
2023-01-03 15:03:19,507 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-03 15:03:19,508 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hotel")
2023-01-03 15:03:19,508 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-03 15:03:19,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-03 15:03:19,510 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotel (
	"index" BIGINT, 
	id BIGINT, 
	ville TEXT, 
	hotel TEXT, 
	url TEXT, 
	note TEXT, 
	description TEXT, 
	latitutde FLOAT, 
	longitude FLOAT
)


2023-01-03 15:03:19,511 INFO sqlalchemy.engine.Engine [no key 0.00068s] ()
2023-01-03 15:03:19,511 INFO sqlalchemy.engine.Engine CREATE INDEX ix_hotel_index ON hotel ("index")
2023-01-03 15:03:19,512 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
2023-01-03 15:03:19,513 INFO sqlalchemy.engine.Engine COMMIT
2023-01-03 15:03:19,515 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-03 15:03:19,

875

In [100]:
df_temps.to_sql("temps", engine)

2023-01-03 15:03:22,002 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("temps")
2023-01-03 15:03:22,002 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-03 15:03:22,003 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("temps")
2023-01-03 15:03:22,004 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-03 15:03:22,005 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-03 15:03:22,006 INFO sqlalchemy.engine.Engine 
CREATE TABLE temps (
	"index" BIGINT, 
	id BIGINT, 
	ville TEXT, 
	lat FLOAT, 
	long FLOAT, 
	temperature_jour_1 FLOAT, 
	temperature_jour_2 FLOAT, 
	temperature_jour_3 FLOAT, 
	temperature_jour_4 FLOAT, 
	temperature_jour_5 FLOAT, 
	temps_jour_1 TEXT, 
	temps_jour_2 TEXT, 
	temps_jour_3 TEXT, 
	temps_jour_4 TEXT, 
	temps_jour_5 TEXT, 
	moyenne FLOAT
)


2023-01-03 15:03:22,006 INFO sqlalchemy.engine.Engine [no key 0.00041s] ()
2023-01-03 15:03:22,007 INFO sqlalchemy.engine.Engine CREATE INDEX ix_temps_index ON temps ("index")
2023-01-03 15:03:22,008 INFO sqlalc

35

In [141]:
conn = engine.connect()

In [142]:
ville = text("SELECT * FROM temps ORDER BY moyenne DESC LIMIT 5")


In [143]:
result_ville = conn.execute(ville)

2023-01-03 15:31:29,798 INFO sqlalchemy.engine.Engine SELECT * FROM temps ORDER BY moyenne DESC LIMIT 5
2023-01-03 15:31:29,798 INFO sqlalchemy.engine.Engine [cached since 1434s ago] ()


In [113]:
result_ville.fetchall()

[(27, 28, 'Uzès', 42.52505, 3.0831554, 12.98, 15.34, 12.16, 12.09, 11.47, 'couvert', 'nuageux', 'légère pluie', 'couvert', 'légère pluie', 12.81),
 (19, 20, 'Montauban', 43.2140359, 5.5396318, 13.4, 11.76, 11.08, 13.58, 12.15, 'nuageux', 'ciel dégagé', 'couvert', 'légère pluie', 'pluie modérée', 12.39),
 (20, 21, 'Toulouse Rive Gauche, Toulouse', 43.2961743, 5.3699525, 12.92, 10.76, 11.08, 13.94, 11.81, 'partiellement nuageux', 'ciel dégagé', 'couvert', 'légère pluie', 'pluie modérée', 12.1),
 (18, 19, 'Biarritz', 43.1506968, 6.3419285, 12.68, 11.73, 10.72, 12.84, 12.2, 'partiellement nuageux', 'ciel dégagé', 'couvert', 'légère pluie', 'légère pluie', 12.03),
 (33, 34, 'Gréoux-les-Bains', 43.4945144, -1.4736657, 13.76, 11.61, 8.74, 10.92, 8.69, 'couvert', 'couvert', 'pluie modérée', 'légère pluie', 'légère pluie', 10.74)]

In [144]:
hotel = text("SELECT * FROM hotel WHERE id IN (28, 20, 21, 19, 34)")

In [145]:
result_hotel = conn.execute(hotel)

2023-01-03 15:31:43,367 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id IN (28, 20, 21, 19, 34)
2023-01-03 15:31:43,368 INFO sqlalchemy.engine.Engine [cached since 466.1s ago] ()


In [150]:
result_hotel.fetchall()

In [146]:
hotel_viz = pd.read_sql_query(hotel, conn)
ville_viz = pd.read_sql_query(ville, conn)

2023-01-03 15:31:47,142 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id IN (28, 20, 21, 19, 34)
2023-01-03 15:31:47,142 INFO sqlalchemy.engine.Engine [cached since 469.9s ago] ()
2023-01-03 15:31:47,145 INFO sqlalchemy.engine.Engine SELECT * FROM temps ORDER BY moyenne DESC LIMIT 5
2023-01-03 15:31:47,146 INFO sqlalchemy.engine.Engine [cached since 1452s ago] ()


In [148]:
ville_viz

,index,id,ville,lat,long,temperature_jour_1,temperature_jour_2,temperature_jour_3,temperature_jour_4,temperature_jour_5,temps_jour_1,temps_jour_2,temps_jour_3,temps_jour_4,temps_jour_5,moyenne
0,27,28,Uzès,42.525050,3.083155,12.98,15.34,12.16,12.09,11.47,couvert,nuageux,légère pluie,couvert,légère pluie,12.81
1,19,20,Montauban,43.214036,5.539632,13.40,11.76,11.08,13.58,12.15,nuageux,ciel dégagé,couvert,légère pluie,pluie modérée,12.39
2,20,21,"Toulouse Rive Gauche, Toulouse",43.296174,5.369953,12.92,10.76,11.08,13.94,11.81,partiellement nuageux,ciel dégagé,couvert,légère pluie,pluie modérée,12.10
3,18,19,Biarritz,43.150697,6.341928,12.68,11.73,10.72,12.84,12.20,partiellement nuageux,ciel dégagé,couvert,légère pluie,légère pluie,12.03
4,33,34,Gréoux-les-Bains,43.494514,-1.473666,13.76,11.61,8.74,10.92,8.69,couvert,couvert,pluie modérée,légère pluie,légère pluie,10.74


In [149]:
hotel_viz

,index,id,ville,hotel,url,note,description,latitutde,longitude
0,450,19,Biarritz,STUDIO Grande Plage de BIARRITZ,https://www.booking.com/hotel/fr/studio-grand-...,"8,0",Vous pouvez bénéficier d'une réduction Genius ...,43.484017,-1.557965
1,451,19,Biarritz,Les Terrasses de Marbella - à 350 m de la plage,https://www.booking.com/hotel/fr/les-terrasses...,"8,0",Doté d'une connexion Wi-Fi et d'un parking pri...,43.468724,-1.570022
2,452,19,"Centre de Biarritz, Biarritz","La Casa Cosy, Appart'Hôtel",https://www.booking.com/hotel/fr/la-casa-cosy-...,"8,8","Situé dans le centre de Biarritz, à 500 mètres...",43.480333,-1.562548
3,453,19,"Centre de Biarritz, Biarritz",Le Gamaritz,https://www.booking.com/hotel/fr/le-gamaritz.f...,"8,6",Vous pouvez bénéficier d'une réduction Genius ...,43.481774,-1.566544
4,454,19,"Centre de Biarritz, Biarritz","Hôtel Littéraire Jules Verne, BW Signature Col...",https://www.booking.com/hotel/fr/karitza-biarr...,"8,5",Vous pouvez bénéficier d'une réduction Genius ...,43.478540,-1.555400
...,...,...,...,...,...,...,...,...,...
120,845,34,Allemagne-en-Provence,Le Petit Grillo - Gite proche des gorges du Ve...,https://www.booking.com/hotel/fr/le-petit-gril...,"9,1","Situé à Allemagne-en-Provence, à 31 km d'ITER ...",43.792566,6.025519
121,846,34,Gréoux-les-Bains,Résidence Les Écureuils,https://www.booking.com/hotel/fr/residence-les...,"9,0","Située à Gréoux-les-Bains, la Résidence Les Éc...",43.753095,5.888536
122,847,34,Moustiers-Sainte-Marie,"La Maison du Courtil, Citron-Caramel",https://www.booking.com/hotel/fr/le-courtil-ap...,"9,3","Situé à Moustiers-Sainte-Marie, à 41 km du par...",43.847565,6.221280
123,848,34,La Palud-sur-Verdon,Auberge des crêtes,https://www.booking.com/hotel/fr/auberge-des-c...,"7,0","Situé à La Palud-sur-Verdon, dans le parc natu...",43.779851,6.355862
